In [2]:
#!pip install siphon metpy

In [3]:
# Import necessary libraries
from datetime import datetime
import numpy as np
from siphon.simplewebservice.wyoming import WyomingUpperAir

In [ ]:

dt = datetime(2024, 9, 30, 12)
station = 'BMX'
df = WyomingUpperAir.request_data(dt, station)
display(df)
p = df['pressure'].values
t = df['temperature'].values
u = df['u_wind'].values
v = df['v_wind'].values

print(p)


In [16]:
from datetime import datetime
import pandas as pd
import numpy as np
from siphon.simplewebservice.wyoming import WyomingUpperAir

# Define the date and time for which you want the data
dt = datetime(2024, 10, 1, 12)  # Change the date as per your requirement

# List of stations
stations = {'Del Rio, Texas': '72261', 'Omaha, Nebraska': '72558', 'International Falls, Minnesota': '72747'}

# Function to filter data between 925 hPa and 500 hPa and export to CSV
def export_filtered_data(df, location):
    # Filter the dataframe for pressures between 925 hPa and 500 hPa
    layer_df = df[(df['pressure'] <= 925) & (df['pressure'] >= 500)]
    # Export the filtered data to a CSV file
    filename = f"{location.replace(',', '').replace(' ', '_')}_pressure_vs_temperature.csv"
    layer_df[['pressure', 'temperature']].to_csv(filename, index=False)
    return filename, layer_df

# Function to calculate mean temperature from CSV using the given logarithmic formula
def calculate_log_mean_temperature_from_csv(filename):
    # Read data from CSV
    df = pd.read_csv(filename)
    pressures = df['pressure']
    temperatures = df['temperature']

    # Calculate the log mean temperature
    sum_log_term = sum(0.5 * (temperatures[i] + temperatures[i+1]) * np.log(pressures[i+1] / pressures[i]) for i in range(len(pressures) - 1))
    total_log = np.log(pressures.iloc[-1] / pressures.iloc[0])

    mean_log_temperature = sum_log_term / total_log if total_log != 0 else None
    return mean_log_temperature

# Loop through the stations and retrieve data
for location, station in stations.items():
    # Request data
    df = WyomingUpperAir.request_data(dt, station)
    
    # Export filtered data to CSV and get the filename and filtered dataframe
    filename, _ = export_filtered_data(df, location)
    
    # Calculate the mean temperature from CSV
    log_mean_temp = calculate_log_mean_temperature_from_csv(filename)
    
    # Print the mean temperature
    if log_mean_temp is not None:
        print(f"{location}: Logarithmic Mean Temperature (925-500 hPa) = {log_mean_temp:.2f}°C")
    else:
        print(f"{location}: Unable to compute logarithmic mean temperature due to insufficient data.")


Del Rio, Texas: Logarithmic Mean Temperature (925-500 hPa) = 8.20°C
Omaha, Nebraska: Logarithmic Mean Temperature (925-500 hPa) = 1.79°C
International Falls, Minnesota: Logarithmic Mean Temperature (925-500 hPa) = -10.25°C
